In [3]:
import numpy as np;
import math;
from enum import Enum;

def load_graph(path):
    edges = np.array([]);
    with open(path, 'r', encoding='utf-8', errors='ignore') as g_file:
        next(g_file); # skip the header line

        for line in g_file:
            try:
                fields = line.split(",");
                edges = np.append(edges, [int(fields[0]), int(fields[1])], axis=None);
                edges = np.reshape(edges, (-1,2))
            except Exception as e:
                pass
    return np.min(edges), np.max(edges), edges;

minedge, maxedge, matrix = load_graph("graph2_edges.txt");
maxedge = int(maxedge);
matrix = np.copy(matrix);
matrix = matrix.astype(int);
print(matrix);
#print(minedge);
#print(maxedge);

prenum = np.full(maxedge+1, 0);
prenumID = 1;

postnum = np.full(maxedge+1, len(matrix));
postnumID = 1;

edges = [];

class EdgeType(Enum):
    TREE = "TREE EDGE";
    FWD = "FORWARD EDGE";
    BACK = "BACK EDGE";
    CROSS = "CROSS EDGE";

class Edge:
    vertexA = None;
    vertexB = None;
    edgeType = None;
    
    def __init__(self, _vertexA, _vertexB, _edgeType):
        self.vertexA = _vertexA;
        self.vertexB = _vertexB;
        self.edgeType = _edgeType;
        
    def isTheSame(self, _vertexA, _vertexB):
        return (_vertexA == vertexA or _vertexA == vertexB) and (_vertexB == vertexA or _vertexB == vertexB);

class Vertex:
    ID = -1;
    directions = [];
    
    def __init__(self, _ID):
        self.ID = _ID;
        self.directions = [];
        
    def addDirection(self, dir):
        #print("ID [" + str(self.ID) + "] appends [" + str(dir.ID) + "]");
        self.directions.append(dir);
    
    def search(self):
        global prenum;
        global prenumID;
        global postnum;
        global postnumID;
        
        prenum[self.ID] = prenumID;
        prenumID += 1;
        
        if(len(self.directions) == 0):
            print("Dead End! at [" + str(self.ID) + "]");
        else:
            for dir in self.directions:                
                if(prenum[dir.ID] == 0):
                    #print("Now in " + str(self.ID) + " going into " + str(dir.ID));
                    #this was tree ege
                    print("[" + str(self.ID) + "] TREE EDGE [" + str(dir.ID) + "]");
                    addEdge(self, dir, EdgeType.TREE);
                    dir.search();
                else:
                   # print("Already visited " + str(dir.ID));
                    Vertex.revisit(self.ID, dir.ID);
        postnum[self.ID] = postnumID;
        postnumID += 1;
    
    def revisit(self, fromID, toVertex):
        global prenum;
        global postnum;
        global postnumID;
        if(prenum[fromID] < prenum[toID]):
            print("[" + str(fromID) + "] FWD EDGE [" + str(toID) + "]");
            addEdge(self, dir, EdgeType.TREE);
        else:
            #cross or back
            #postFromSmaller = False;
            #if(postnum[toID] == math.inf):
            #    print("Didn't return from there yet");
            #    postFromSmaller = prenum[fromID] > prenum[toID];
            #else:
            #    postFromSmaller = postnum[fromID] < postnum[toID]
            
            if(postnumID < postnum[toID]):
                print("[" + str(fromID) + "] BACK EDGE [" + str(toID) + "]");
                addEdge(self, dir, EdgeType.TREE);
            else:
                print("[" + str(fromID) + "] CROSS EDGE [" + str(toID) + "]");
                addEdge(self, dir, EdgeType.TREE);
        

def addEdge(vertexA, vertexB, edgeType):
    global edges;
    isNewEdge = True;
    for edge in edges:
        isNewEdge = isNewEdge and not edge.isTheSame(vertexA, vertexB);
        if (not isNewEdge):
            print("Edge between " + str(vertexA.ID) + " and " + str(vertexB.ID) + " was already defined!");
            break;
    if (isNewEdge):
        print("Created a new edge!");
        edges.append(Edge(vertexA, vertexB, edgeType));
        
vertices = [None]*len(matrix); 
for i in range(len(matrix)):
    vertices[i] = Vertex(i);
    
#for v in vertices:
#    print("VertexID: " + str(v.ID));

for i in range(len(matrix)):
    vID = matrix[i,0];
    dID = matrix[i,1];
    vertices[vID].addDirection(vertices[dID]);
    
#for vertex in vertices:
#    print("------" + str(v.ID) + "------");
#    for v in vertex.directions:
#        print(v.ID);
        
print("-----Starting the search!-----");
vertices[0].search();
print("-------Search is done!--------\n\n");
print("PreNum:");
print(prenum);
print("PostNum:");
print(postnum);

[[0 1]
 [1 0]
 [1 2]
 [2 1]
 [2 0]
 [0 2]
 [1 3]
 [3 1]
 [3 4]
 [4 3]
 [4 0]
 [0 4]
 [1 5]
 [5 1]
 [5 6]
 [6 5]
 [6 1]
 [1 6]
 [1 7]
 [7 1]
 [7 8]
 [8 7]
 [8 9]
 [9 8]
 [8 1]
 [1 8]]
-----Starting the search!-----
[0] TREE EDGE [1]
[1] BACK EDGE [0]
[1] TREE EDGE [2]
[2] BACK EDGE [1]
[2] BACK EDGE [0]
[1] TREE EDGE [3]
[3] BACK EDGE [1]
[3] TREE EDGE [4]
[4] BACK EDGE [3]
[4] BACK EDGE [0]
[1] TREE EDGE [5]
[5] BACK EDGE [1]
[5] TREE EDGE [6]
[6] BACK EDGE [5]
[6] BACK EDGE [1]
[1] FWD EDGE [6]
[1] TREE EDGE [7]
[7] BACK EDGE [1]
[7] TREE EDGE [8]
[8] BACK EDGE [7]
[8] TREE EDGE [9]
[9] BACK EDGE [8]
[8] BACK EDGE [1]
[1] FWD EDGE [8]
[0] FWD EDGE [2]
[0] FWD EDGE [4]
-------Search is done!--------


PreNum:
[ 1  2  3  4  5  6  7  8  9 10]
PostNum:
[10  9  1  3  2  5  4  8  7  6]
